# Load libraries

In [ ]:
from dingo import MetabolicNetwork, PolytopeSampler
from dingo.utils import correlated_reactions

import numpy as np
import pickle
from cobra.io import load_json_model, read_sbml_model, load_matlab_model
import cobra

from scipy import stats
from statsmodels.stats.multitest import multipletests

import matplotlib.pyplot as plt

# Load models

In [ ]:
roseburia_dingo_model = MetabolicNetwork.from_mat("/home/touliopoulos/project/erasmus/erasmus_2025_project/data/Reconstructions_data/Roseburia_intestinalis_L1_82_fixed.mat")
roseburia_cobra_model = load_matlab_model("/home/touliopoulos/project/erasmus/erasmus_2025_project/data/Reconstructions_data/Roseburia_intestinalis_L1_82_fixed.mat")
reactions = roseburia_dingo_model.reactions


# Load sampling data

In [ ]:



with open("/home/touliopoulos/project/erasmus/erasmus_2025_project/data/Steady_States_Samples/RI_no_mucins/RI_steady_states_opt_0_rep_002_subsampled.pckl", "rb") as absolute_steady_states_0:
    absolute_steady_states_0 = abs(pickle.load(absolute_steady_states_0))

with open("/home/touliopoulos/project/erasmus/erasmus_2025_project/data/Steady_States_Samples/RI_no_mucins/RI_steady_states_opt_1_rep_002_subsampled.pckl", "rb") as absolute_steady_states_1:
    absolute_steady_states_1 = abs(pickle.load(absolute_steady_states_1))
    
with open("/home/touliopoulos/project/erasmus/erasmus_2025_project/data/Steady_States_Samples/RI_no_mucins/RI_steady_states_opt_99_rep_002_subsampled.pckl", "rb") as absolute_steady_states_99:
    absolute_steady_states_99 = abs(pickle.load(absolute_steady_states_99))

with open("/home/touliopoulos/project/erasmus/erasmus_2025_project/data/Steady_States_Samples/RI_no_mucins/RI_steady_states_opt_100_rep_002_subsampled.pckl", "rb") as absolute_steady_states_100:
    absolute_steady_states_100 = abs(pickle.load(absolute_steady_states_100))



# Perform statistical analysis

In [ ]:

conditions = [
    absolute_steady_states_001,
    absolute_steady_states_002,
    absolute_steady_states_003
]


# Add a fold change ?


selected_comparisons = [(0, 1), (0, 2), (1, 2)]
#selected_comparisons = [(0, 1)]


p_values_mw = {row: [] for row in range(len(reactions)) }  # Store p-values for each row for MW test
p_values_ks = {row: [] for row in range(len(reactions)) }  # Store p-values for each row for KS test
values_mw = {row: [] for row in range(len(reactions)) }  # Store mw-values for each row for MW test
values_ks = {row: [] for row in range(len(reactions)) }  # Store ks-values for each row for KS test

for row in range(len(reactions)):  # Iterate over rows
    for i, j in selected_comparisons:
        
        mw, p_mw = stats.mannwhitneyu(conditions[i][row], conditions[j][row], alternative='two-sided')
        mw = mw / (len(conditions[i][row]) * len(conditions[j][row]))
        
        ks, p_ks = stats.ks_2samp(conditions[i][row], conditions[j][row], alternative='two-sided')

        p_values_mw[row].append(p_mw)
        p_values_ks[row].append(p_ks)
        
        values_mw[row].append(mw)
        values_ks[row].append(ks)

In [ ]:
print(p_values_ks)
p_values_ks_copy = list(p_values_ks.values())
flat_p_values_ks = np.array(p_values_ks_copy).flatten()

# Apply FDR correction
rejected, corrected_p_values_ks, _, _ = multipletests(flat_p_values_ks, method='fdr_bh')

# Reshape corrected p-values back to the original matrix shape
corrected_p_values_ks = corrected_p_values_ks.reshape(np.array(p_values_ks_copy).shape)
print(corrected_p_values_ks)



# for KS
raw_p = [p for row in p_values_ks.values() for p in row]
#corrected_p = [p for row in corrected_p_values_ks.values() for p in row]
corrected_p = corrected_p_values_ks.flatten()

plt.scatter(raw_p, corrected_p)
plt.xlabel("Raw p-values")
plt.ylabel("Corrected p-values")
plt.title("Raw vs. Corrected p-values")
#plt.xscale("log")
#plt.yscale("log")
plt.show()

In [ ]:


cobra_reactions = roseburia_cobra_model.reactions
reaction_dict = {str(reaction.id): str(reaction.name) for reaction in cobra_reactions if "acetat" in str(reaction.name).lower() or "lactat" in str(reaction.name).lower()}

for a, b, c, d in zip(filtered_reactions, mean_differences_steady_states, std_differences_steady_states, mean_differences_corr_matrix):
    if a in reaction_dict.keys():
        print(a, reaction_dict[a], b, c, d)
        